# Machine Learning prediction of heatwaves

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
from pathlib import Path

import utilities as ut

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject


# Data

## Fetch the data

In [2]:
data_info_dict = ut.json2dict('data_info.json')

def fetch(name, destination='./data', reload=False):
    destination = Path(destination).resolve()
    if not os.path.exists(destination):
        destination.mkdir(parents=True)

    if name.endswith('.nc'):
        name = name[:-3]

    file = destination / f'{name}.nc'
    if not os.path.exists(file) or reload:
        try:
            link = data_info_dict[name]['link']
        except KeyError:
            raise KeyError(f'No item {name} in data_info.json')
        os.system(f"wget -O {file} \"{link}\"")
    else:
        print('file already loaded')

## Look at the data

## Compute the labels

## Prepare the fields to feed to the network

# Neural networks

## Build a neural network

## Training a neural network

## K-fold cross validation

# Hyperparameter optimization